# Biocrnpyler Rheostat Attempt 2
5.14.2020

In this notebook, we will try to implement simplified model 1 from bioscrape into biocrnpyler.

Let's write a plan of action:<br>
1) **make a new Mechanism** <br>
  - call it FuelMichaelisMenten with type 'catalysis' <br>
    - F+S+E <--> C1 --> C2 <--> W+P+E <br>
    - should take a list: Substrates = [s1, s2, ..], Enzyme = E, products = [p1, p2, ..] <br>
    - num substrate != num product. multiples of same species can appear. Maybe call it GeneralizedMichaelisMenten <br>
    
2) **make a new Component**<br>
  - call it 'Enzyme' <br>
    - contains a 'catalysis' Mechanism  <br>
    - contains internal species (passed in for the constructor) which represents enzyme's name, substrate, fuel, product, waste<br>
    
3) **make a new Mixture**<br>
 - call it EnergyTxTl <br>
    - has a default 'catalysis' Mechanism which is FuelMichaelisMenten<br>
    - can be a subclass of the TxTl mixture <br>
    
4) **test all the parts together** to see if they mimic bioscrape <br>

- each enzyme needs to be its own component so we can vary the concentrations and rates in a systematic way easily <br>

**References:** <br>
- biocrnpyler lecture slides <br>
- Developer Overview ipython notebook for examples on how to create Mechanisms, Components, and Mixtures


In [65]:
# import cell
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
#import bokeh_catplot
bokeh.io.output_notebook()

from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
from biocrnpyler import Mixture
from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Loading BokehJS ...

In [66]:
def check_type(item, material_type_str):
    if isinstance(item, Species):
        item_ret = item
    elif isinstance(item, str):
        item_ret = Species(name = item, material_type = material_type_str)
    elif isinstance(item, Component) and item.get_species() != None:
        item_ret = item.get_species()
    else:
        raise ValueError( item, "parameter must be a string, a Component with defined get_species(), or a chemical_reaction_network.species")
        
    return item_ret

## 1) Make a new Mechanism: FuelMichaelisMenten

In [90]:
# Make a new mechanism - black box, contain all the intermediate steps required to get from input to output
# rewrite object constructor: set name of mechanism and name of enzyme species, enz
# rewrite update_species: create a list of all the species used in the reaction schema: including fuel, substrate, enzyme
    # f:s:e complex, w:p:e complex, and product, waste
# rewite update_reactions: create a list of all the reactions required for reaction schema: f, s, e binding, fse to wpe, 
    # and wpe unbinding / wp production

class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, enzyme, type = 'catalysis', **keywords):
        
        self.enzyme = check_type(enzyme, 'enzyme')
   
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)
        
    # overwrite update_species
    #  product : the name of the output product
    #  waste : the name of the output waste
    def update_species(self, fuel, substrate, product, waste):
        
        species = [fuel, substrate, product, waste, self.enzyme]
        species += [ComplexSpecies([self.enzyme,fuel, substrate])]
        species += [ComplexSpecies([self.enzyme, waste, product])] # IS THIS LINE NECESSARY
        return species
    
    # overwrite update_reactions
    #    requires rates and relevant species. Returns a list of chemical_reaction_network.reaction
    def update_reactions(self, fuel, substrate, product, waste, component, part_id = None):
        
        # parameters
#         k_bf = component.get_parameter("k_bf", part_id = part_id, mechanism = self)
#         k_br = 0.1 * k_bf
#         k_uf = component.get_parameter("k_uf", part_id = part_id, mechanism = self)
#         k_ur = 0.1 * k_uf
#         k_cat = component.get_parameter("k_cat", part_id = part_id, mechanism = self)
#         # Catalysis rate of all other reactions that use atp
#         k_atp_use = component.get_parameter("k_atp_use", part_id = part_id, mechanism = self)
        
        k_bf = 22.68
        k_br = 2.268
       # k_br = 0.1 * k_bf
        k_uf = 24
        k_ur = 2.4
       # k_ur = 0.1 * k_uf
        k_cat = 10
        # Catalysis rate of all other reactions that use atp
        k_atp_use = 0.5
        
        
        comp1 = ComplexSpecies([self.enzyme,fuel, substrate])
        comp2 = ComplexSpecies([self.enzyme, waste, product])
        
        binding_rxn = Reaction(inputs = [fuel, substrate, self.enzyme], outputs=[comp1], k = k_bf, k_rev = k_br)
        cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
        unbinding_rxn = Reaction(inputs = [comp2], outputs = [waste, product, self.enzyme], k=k_uf, k_rev = k_ur)
        
        return [binding_rxn, cat_rxn, unbinding_rxn]

In [86]:
# class FuelMichaelisMenten(Mechanism):
#     def __init__(self, name, type = 'catalysis', **keywords):
    
        
#         Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)

#     def update_species(self, enzyme, fuel, substrate, product, waste): 
        
#         self.enzyme = check_type(enzyme, 'enzyme')
#         species = [self.enzyme, fuel, substrate, product, waste]
        
#         comp1 = ComplexSpecies(self.enzyme, fuel, substrate)
#         comp2 = ComplexSpecies(self.enzyme, product, waste)
       
        
#         species += [ComplexSpecies(comp1)]
#         species += [ComplexSpecies(comp2)]
#         return species
    
#     def update_reactions(self, enzyme, fuel, substrate, product, waste, k_cat, component, part_id = None):

#         k_bf = 22.68
        
#         #k_bf = component.get_parameter('k_bf', part_id = part_id, mechanism = self)
        
#         k_br = 2.268
#         k_uf = 24
#         k_ur = 2.4
#        # k_cat = 10
#        # k_atp_use = 0.5
#        # print(enzyme)
#         self.enzyme = check_type(enzyme, 'enzyme')
        
        
#         comp1 = ComplexSpecies(self.enzyme, fuel, substrate)
#         comp2 = ComplexSpecies(self.enzyme, product, waste)
        
#         binding_rxn = Reaction(inputs = [self.enzyme, fuel, substrate], outputs=[comp1], k = k_bf, k_rev = k_br)
#         cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
#         unbinding_rxn = Reaction(inputs = [comp2], outputs = [self.enzyme, product, waste], k=k_uf, k_rev = k_ur)
        
#         return [binding_rxn, cat_rxn, unbinding_rxn]

    
## 2) Make a new Component: Enzyme
  - call it 'Enzyme' <br>
    - contains a 'catalysis' Mechanism  <br>
    - contains internal species (passed in for the constructor) which represents enzyme's name, substrate, fuel, product, waste<br>

In [87]:
#self.internal_species = self.set_species(species, ....)

In [46]:
def list_input(item, material_type_str):
    return check_type(self, item, 'material_type_str')

In [91]:
# Make a new component
# rewrite constructor: set name of enzyme species and name of product
# rewrite update_species: call each mechanisms (FuelMichaelisMenten) to get species
# rewrite update_reactions: call each mechanisms (FuelMichaelisMenten) to get reactions
class Enzyme(Component):
    def __init__(self, enzyme_name, substrate, fuel, product, waste, **keywords):
      
        # ENZYME NAME
        self.enzyme = check_type(enzyme_name, 'enzyme')
    
        # SUBSTRATE
        self.substrate = check_type(substrate, 'molecule')
        
#         # FUEL
        self.fuel = check_type(fuel, 'metabolite')
#         for f in fuel:
#             setattr(self,f'{f}',check_type(f, 'metabolite'))
        
#         getattr(self,f'{f}')
                  
            
#         self.fuel_0 = check_type(fuel[0], 'metabolite')
#         self.fuel_1 = check_type(fuel[1], 'metabolite')
#     #    self.fuel = check_type(self, fuel, 'metabolite')
    
#         # PRODUCT
        self.product = check_type(product, 'molecule')
#         self.product_0 = check_type(product[0], 'molecule')
#         self.product_1 = check_type(product[1], 'molecule')


            
        # WASTE
        self.waste = check_type(waste, 'metabolite')
      
        Component.__init__(self = self, name = enzyme_name, **keywords)
        
    def update_species(self):
        #mech_express = self.mechanisms["gene_expression"]
        mech_cat = self.mechanisms['catalysis']
        
        # NOTE self.enzyme not included below
        return mech_cat.update_species(self.fuel, self.substrate, self.product, self.waste) 
                                                                                           
    
    def update_reactions(self):
        #mech_express = self.mechanisms["gene_expression"]
        mech_cat = self.mechanisms['catalysis']
        
        return mech_cat.update_reactions(self.fuel, self.substrate, self.product, self.waste,
                                             component = self )
    #part_id = self.name

In [85]:
# class Enzyme(Component):
#     def __init__(self, enzyme_name, substrate, fuel, product, waste, k_cat = 10, **keywords):
      
#         # ENZYME NAME
#         # self.enzyme = self.set_species()
#         self.enzyme = check_type(enzyme_name, 'enzyme')
    
#         # SUBSTRATE
#         self.substrate = self.set_species(substrate, material_type = 'molecule')
     
#         # FUEL
        
#         self.fuel = self.set_species(fuel, material_type = 'metabolite')
    
        
#         # PRODUCT
#         self.product = self.set_species(product, material_type = 'molecule')
    
#         # WASTE
#         self.waste = self.set_species(waste, material_type = 'metabolite')
      
#         self.k_cat = k_cat
            
        
      
#         Component.__init__(self = self, name = enzyme_name, **keywords)
        
#     def update_species(self):
#         mech_cat = self.mechanisms['catalysis']
        
            
        
#         return mech_cat.update_species(self.enzyme, self.fuel, self.substrate, self.product, self.waste) 
                                                                                           
    
#     def update_reactions(self):
#         mech_cat = self.mechanisms['catalysis']

        
#         return mech_cat.update_reactions(self.enzyme, self.fuel, self.substrate, self.product, self.waste,self.k_cat,
#                                              component = self, part_id = self.name) # for parameters



## 3) Make a new Mixture: EnergyTxTl
 - call it EnergyTxTl <br>
    - has a default 'catalysis' Mechanism which is FuelMichaelisMenten<br>
    - can be a subclass of the TxTl mixture <br>

In [92]:
#ExpressionMixture
from biocrnpyler import Mixture

class EnergyTxTl(Mixture):
    #OVERWRITE THIS METHOD
    def __init__(self, name="",enzyme = 'enzyme',**keywords): # instead of 'enzyme'
    #def __init__(self, name="", **keywords):
        
        self.enzyme = check_type(enzyme, 'enzyme')

        mech_cat = FuelMichaelisMenten('catalysis', self.enzyme) # took out, self.enzyme
        
        default_mechanisms = {
            mech_cat.mechanism_type:mech_cat
        }
        
        species = [self.enzyme]
        Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords)        
    

**Need to create a mixture suitable for your enzymes that had default mechanisms for them.** <br>
Finally, in energy mixture, you don't need an enzyme keyword - eventually you will add many enzymes with multiple fuels as components etc.<br>
Instead, add Enzymes to the Mixture as Components

In [69]:
# class EnergyTxTl(Mixture):
#     def __init__(self, name="",**keywords): 
        

#         mech_cat = FuelMichaelisMenten('catalysis')
        
#         default_mechanisms = {
#             mech_cat.mechanism_type:mech_cat
#         }
        
#         #species = [self.enzyme]
#         Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords) 
#         #  parameter_file = 'parameters.txt'
    

In [93]:
#Create a fake parameter dictionary for the example

# parameters = {("catalysis","enzyme", "k_bf"):1.0, 
#               ("catalysis","enzyme", "k_uf"):0.1,
#               ("catalysis","enzyme", "k_cat"):100.0,
#               ("catalysis","enzyme", "k_atp"):100.0}

#Instantiate an enzyme
E1 = Enzyme(enzyme_name= "enzyme1", substrate = "glucose", 
            fuel = '2atp',product = 'f16p', waste = '2adp') #, parameters = parameters)
            #enzyme_name, substrate, fuel, product, waste,
    
E2 = Enzyme(enzyme_name = 'enzyme2', substrate = 'f16p', fuel = '2adp',
           product = 'isobutanol', waste = '2atp')

E3 = Enzyme(enzyme_name = 'enzyme3', substrate = 'f16p', fuel = '2adp', product = 'isobutanol',
           waste = '2atp')

E4 = Enzyme(enzyme_name = 'enzyme4', substrate = 'sub', fuel = '2atp', product = 'prod', waste = '2adp')

myMixture = EnergyTxTl(components = [E1])
CRN = myMixture.compile_crn()

myMixture_atp = EnergyTxTl(components = [E4])
CRN_atp = myMixture_atp.compile_crn()
print(CRN.pretty_print(show_rates =False ))

Species (7) = {0. metabolite[2atp], 1. molecule[glucose], 2. molecule[f16p], 3. metabolite[2adp], 4. enzyme[enzyme], 5. complex[enzyme[enzyme]:metabolite[2atp]:molecule[glucose]], 6. complex[enzyme[enzyme]:metabolite[2adp]:molecule[f16p]]}
Reactions (3) = [
0. metabolite[2atp] + molecule[glucose] + enzyme[enzyme] <--> complex[enzyme[enzyme]:metabolite[2atp]:molecule[glucose]]        
1. complex[enzyme[enzyme]:metabolite[2atp]:molecule[glucose]] --> complex[enzyme[enzyme]:metabolite[2adp]:molecule[f16p]]        
2. complex[enzyme[enzyme]:metabolite[2adp]:molecule[f16p]] <--> metabolite[2adp] + molecule[f16p] + enzyme[enzyme]        
]


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.


**Need to create a mixture suitable for your enzymes that had default mechanisms for them.**

In [94]:
CRN.write_sbml_file("CRN.sbml")
CRN_atp.write_sbml_file("CRN_atp.sbml")
print(CRN.pretty_print(show_rates = False))

Species (7) = {0. metabolite[2atp], 1. molecule[glucose], 2. molecule[f16p], 3. metabolite[2adp], 4. enzyme[enzyme], 5. complex[enzyme[enzyme]:metabolite[2atp]:molecule[glucose]], 6. complex[enzyme[enzyme]:metabolite[2adp]:molecule[f16p]]}
Reactions (3) = [
0. metabolite[2atp] + molecule[glucose] + enzyme[enzyme] <--> complex[enzyme[enzyme]:metabolite[2atp]:molecule[glucose]]        
1. complex[enzyme[enzyme]:metabolite[2atp]:molecule[glucose]] --> complex[enzyme[enzyme]:metabolite[2adp]:molecule[f16p]]        
2. complex[enzyme[enzyme]:metabolite[2adp]:molecule[f16p]] <--> metabolite[2adp] + molecule[f16p] + enzyme[enzyme]        
]


In [95]:
timepoints = np.linspace(0,70,100)
x0 = {"enzyme_enzyme":0.15,
     'molecule_glucose':15,
     'metabolite_2atp': 15,
     'molecule_sub':5}

x0_atp = {"enzyme_enzyme":0.15,
     'metabolite_2atp': 15,
     'molecule_sub':5}

re, me= CRN.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0, file = 'CRN.sbml')
re_atp,me_atp = CRN_atp.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0_atp, file = 'CRN_atp.sbml')

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bioscrape/sbmlutil.py:209: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/chemical_reaction_network.py:900: UserWarning: Trying to set species that is not in model: molecule_sub
  m.set_species(initial_condition_dict)


In [96]:
p1 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn 1,2,3', x_axis_label = 'time',
                          y_axis_label = 'concentration')
p1.line(timepoints, re['metabolite_2atp'], legend_label = 'atp', color = 'red')
p1.line(timepoints, re['molecule_glucose'], legend_label = 'glucose', color = 'green')
p1.line(timepoints, re['enzyme_enzyme'], legend_label = 'enzyme', color = 'blue')
p1.line(timepoints, re['metabolite_2adp'], legend_label = 'adp', color = 'pink')
p1.line(timepoints, re['molecule_f16p'], legend_label = 'f16p', color = 'purple')
#p1.line(timepoints, re['molecule_isobutanol'], legend_label = 'isobutanol', color = 'orange')

p2 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn atp leak only', x_axis_label = 'time',
                          y_axis_label = 'concentration', y_range = p1.y_range)
#p2.line(timepoints, re_atp['metabolite_2atp'], legend_label = 'atp', color = 'red')
#p2.line(timepoints, re_atp['metabolite_2adp'], legend_label = 'adp', color = 'pink')

bokeh.io.show(row(p1, p2))

In [20]:
re

,metabolite_2atp,molecule_glucose,molecule_f16p,metabolite_2adp,enzyme_enzyme,complex_enzyme_enzyme_metabolite_2atp_molecule_glucose,complex_enzyme_enzyme_metabolite_2adp_molecule_f16p,molecule_isobutanol,complex_enzyme_enzyme_metabolite_2atp_molecule_isobutanol,molecule_sub,molecule_prod,complex_enzyme_enzyme_metabolite_2atp_molecule_sub,complex_enzyme_enzyme_metabolite_2adp_molecule_prod,time
0,15.000000,1.500000e+01,0.000000e+00,0.000000,0.150000,0.000000e+00,0.000000e+00,0.000000,0.000000,5.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
1,14.313668,1.430859e+01,3.461466e-01,0.536556,0.000224,8.482787e-02,1.751654e-02,0.235547,0.007371,4.769530e+00,0.190409,2.827596e-02,0.011785,0.707071
2,13.763113,1.370953e+01,6.969592e-01,1.087129,0.000243,8.465424e-02,1.761812e-02,0.483742,0.007498,4.569843e+00,0.390170,2.821808e-02,0.011769,1.414141
3,13.216883,1.311216e+01,1.043972e+00,1.633380,0.000263,8.437452e-02,1.781814e-02,0.733998,0.007674,4.370721e+00,0.589408,2.812484e-02,0.011746,2.121212
4,12.677619,1.251734e+01,1.384816e+00,2.172667,0.000286,8.396827e-02,1.813587e-02,0.987835,0.007907,4.172446e+00,0.787851,2.798942e-02,0.011713,2.828283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9.878713,1.332440e-13,3.580453e-11,4.979539,0.008252,2.279393e-14,1.476244e-12,14.878713,0.121287,4.441466e-14,4.979539,7.597977e-15,0.020461,67.171717
96,9.878713,1.749002e-13,3.822744e-11,4.979539,0.008252,2.992001e-14,1.577266e-12,14.878713,0.121287,5.830006e-14,4.979539,9.973336e-15,0.020461,67.878788
97,9.878713,2.060603e-13,3.965747e-11,4.979539,0.008252,3.525052e-14,1.637119e-12,14.878713,0.121287,6.868675e-14,4.979539,1.175017e-14,0.020461,68.585859
98,9.878713,2.267242e-13,4.009463e-11,4.979539,0.008252,3.878548e-14,1.655801e-12,14.878713,0.121287,7.557472e-14,4.979539,1.292849e-14,0.020461,69.292929


In [14]:
re

,metabolite_2atp,molecule_glucose,molecule_f16p,metabolite_2adp,enzyme_enzyme,complex_enzyme_enzyme_metabolite_2atp_molecule_glucose,complex_enzyme_enzyme_metabolite_2adp_molecule_f16p,molecule_isobutanol,complex_enzyme_enzyme_metabolite_2atp_molecule_isobutanol,molecule_sub,molecule_prod,complex_enzyme_enzyme_metabolite_2atp_molecule_sub,complex_enzyme_enzyme_metabolite_2adp_molecule_prod,time
0,15.000000,1.500000e+01,0.000000e+00,0.000000,0.150000,0.000000e+00,0.000000e+00,0.000000,0.000000,5.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
1,14.313668,1.430859e+01,3.461466e-01,0.536556,0.000224,8.482787e-02,1.751654e-02,0.235547,0.007371,4.769530e+00,0.190409,2.827596e-02,0.011785,0.707071
2,13.763113,1.370953e+01,6.969592e-01,1.087129,0.000243,8.465424e-02,1.761812e-02,0.483742,0.007498,4.569843e+00,0.390170,2.821808e-02,0.011769,1.414141
3,13.216883,1.311216e+01,1.043972e+00,1.633380,0.000263,8.437452e-02,1.781814e-02,0.733998,0.007674,4.370721e+00,0.589408,2.812484e-02,0.011746,2.121212
4,12.677619,1.251734e+01,1.384816e+00,2.172667,0.000286,8.396827e-02,1.813587e-02,0.987835,0.007907,4.172446e+00,0.787851,2.798942e-02,0.011713,2.828283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9.878713,1.332440e-13,3.580453e-11,4.979539,0.008252,2.279393e-14,1.476244e-12,14.878713,0.121287,4.441466e-14,4.979539,7.597977e-15,0.020461,67.171717
96,9.878713,1.749002e-13,3.822744e-11,4.979539,0.008252,2.992001e-14,1.577266e-12,14.878713,0.121287,5.830006e-14,4.979539,9.973336e-15,0.020461,67.878788
97,9.878713,2.060603e-13,3.965747e-11,4.979539,0.008252,3.525052e-14,1.637119e-12,14.878713,0.121287,6.868675e-14,4.979539,1.175017e-14,0.020461,68.585859
98,9.878713,2.267242e-13,4.009463e-11,4.979539,0.008252,3.878548e-14,1.655801e-12,14.878713,0.121287,7.557472e-14,4.979539,1.292849e-14,0.020461,69.292929


Mechanism leftover

In [16]:
# Make a new mechanism - black box, contain all the intermediate steps required to get from input to output
# rewrite object constructor: set name of mechanism and name of enzyme species, enz
# rewrite update_species: create a list of all the species used in the reaction schema: including fuel, substrate, enzyme
    # f:s:e complex, w:p:e complex, and product, waste
# rewite update_reactions: create a list of all the reactions required for reaction schema: f, s, e binding, fse to wpe, 
    # and wpe unbinding / wp production

class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, enzyme, type = 'catalysis', **keywords):
        
        self.enzyme = check_type(self, enzyme, 'enzyme')
        
        
    #def __init__(self, name, enzyme, type = 'catalysis'):
#         if isinstance(enzyme, Species):
#             self.enzyme = enzyme
#         elif isinstance(enzyme, str):
#             self.enzyme = Species(name = enzyme, material_type = 'enzyme')
#         elif isinstance(enzyme, Component) and enzyme.get_species() != None:
#             self.enzyme = enzyme.get_species()
#         else:
#             raise ValueError("'enzyme' parameter must be a string, a Component with defined get_specie() or a chemical_reaction_network.specie")
        
        
        #self.enzyme = self.set_species(enzyme, material_type = 'enzyme')
        
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)
        
    # overwrite update_species
    #  product : the name of the output product
    #  waste : the name of the output waste
    def update_species(self, fuel, substrate, product, waste):
        species = [fuel, substrate, product, waste, self.enzyme]
        species += [ComplexSpecies([self.enzyme,fuel, substrate])]
        species += [ComplexSpecies([self.enzyme, waste, product])] # IS THIS LINE NECESSARY
        return species
    
    # overwrite update_reactions
    #    requires rates and relevant species. Returns a list of chemical_reaction_network.reaction
    def update_reactions(self, fuel, substrate, product, waste, component, part_id = None):
        
        # parameters
#         k_bf = component.get_parameter("k_bf", part_id = part_id, mechanism = self)
#         k_br = 0.1 * k_bf
#         k_uf = component.get_parameter("k_uf", part_id = part_id, mechanism = self)
#         k_ur = 0.1 * k_uf
#         k_cat = component.get_parameter("k_cat", part_id = part_id, mechanism = self)
#         # Catalysis rate of all other reactions that use atp
#         k_atp_use = component.get_parameter("k_atp_use", part_id = part_id, mechanism = self)
        
        k_bf = 100
        k_br = 0.1 * k_bf
        k_uf = 100
        k_ur = 0.1 * k_uf
        k_cat = 1000
        # Catalysis rate of all other reactions that use atp
        k_atp_use = 1000
        
#         k_bf = component.get_parameter("kb", part_id = part_id, mechanism = self)
#         k_br = 0.1 * k_bf
#         k_uf = component.get_parameter("ku", part_id = part_id, mechanism = self)
#         k_ur = 0.1 * k_uf
#         k_cat = component.get_parameter("kexpress", part_id = part_id, mechanism = self)
#         # Catalysis rate of all other reactions that use atp
#         k_atp_use = component.get_parameter("kexpress", part_id = part_id, mechanism = self)
        
        
        
        comp1 = ComplexSpecies([self.enzyme,fuel, substrate])
        comp2 = ComplexSpecies([self.enzyme, waste, product])
        
        binding_rxn = Reaction(inputs = [fuel, substrate, self.enzyme], outputs=[comp1], k = k_bf, k_rev = k_br)
        cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
        unbinding_rxn = Reaction(inputs = [comp2], outputs = [waste, product, self.enzyme], k=k_uf, k_rev = k_ur)
        
        return [binding_rxn, cat_rxn, unbinding_rxn]